# Persiapan Awal
Instalasi library yang diperlukan untuk proyek ini:

In [ ]:
!pip install scikit-image tqdm seaborn

# Import Library
Mengimpor semua library yang akan digunakan dalam analisis:

In [ ]:
import os
import cv2
import numpy as np
from skimage import filters
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

# Konfigurasi Dataset
Menentukan parameter dasar untuk dataset ekspresi wajah:

In [ ]:
data_dir = '/kaggle/input/dataset-ekspresi/dataset-ekspresi'
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
img_size = 48

print("Memulai pelatihan model deteksi emosi dengan Gabor filter...")

# Augmentasi Data
Fungsi untuk menambah variasi data training dengan berbagai transformasi:

In [ ]:
def augment_image(image):
    augmented_images = [image]
    # Skala berbeda
    for scale in [0.8, 1.2]:
        scaled = cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        scaled = cv2.resize(scaled, (img_size, img_size))
        augmented_images.append(scaled)
    # Rotasi
    for angle in [-15, 15]:
        M = cv2.getRotationMatrix2D((img_size/2, img_size/2), angle, 1)
        rotated = cv2.warpAffine(image, M, (img_size, img_size))
        augmented_images.append(rotated)
    # Flip horizontal
    flipped = cv2.flip(image, 1)
    augmented_images.append(flipped)
    # Variasi kecerahan
    for alpha in [0.8, 1.2]:
        bright = cv2.convertScaleAbs(image, alpha=alpha, beta=0)
        augmented_images.append(bright)
    # Tambahkan noise
    noise = np.random.normal(0, 25, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    augmented_images.append(noisy_image)
    return augmented_images

# Ekstraksi Fitur Gabor
Menggunakan Gabor filter untuk mengekstrak ciri tekstur dari gambar:

In [ ]:
def extract_gabor_features(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) == 3 else image
    gabor_features = []
    # Ekstraksi dengan berbagai orientasi
    for theta in [0, np.pi/4, np.pi/2, 3*np.pi/4]:
        gabor = filters.gabor(image, frequency=0.2, theta=theta)[1]
        gabor_features.append(gabor.flatten())
    return np.concatenate(gabor_features)

# Memuat Dataset
Proses loading data dan ekstraksi fitur untuk semua gambar:

In [ ]:
def load_dataset():
    data = []
    labels = []
    total_images = sum(len(os.listdir(os.path.join(data_dir, emotion))) for emotion in emotions)
    with tqdm(total=total_images, desc="Memuat dataset Gabor", unit="gambar") as pbar:
        for emotion in emotions:
            folder = os.path.join(data_dir, emotion)
            for img_name in os.listdir(folder):
                img_path = os.path.join(folder, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, (img_size, img_size))
                    augmented_imgs = augment_image(img)
                    for aug_img in augmented_imgs:
                        features = extract_gabor_features(aug_img)
                        data.append(features)
                        labels.append(emotion)
                pbar.update(1)
    return np.array(data), np.array(labels)

# Persiapan Data
Memuat data dan melakukan encoding label:

In [ ]:
# Memuat data
with tqdm(total=1, desc="Memproses dataset", unit="dataset") as pbar:
    X_gabor, y_gabor = load_dataset()
    pbar.update(1)

# Encoding label
with tqdm(total=1, desc="Mengencode label", unit="task") as pbar:
    label_encoder = LabelEncoder()
    y_gabor_encoded = label_encoder.fit_transform(y_gabor)
    y_gabor = to_categorical(y_gabor_encoded)
    pbar.update(1)

# Arsitektur Model
Membangun model neural network untuk klasifikasi:

In [ ]:
def create_model(input_shape, num_classes):
    model = Sequential([
        Dense(1024, activation='relu', input_shape=(input_shape,)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.4),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

# Visualisasi Pelatihan
Fungsi untuk memplot akurasi dan loss selama training:

In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Gabor Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Gabor Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Evaluasi Model
Fungsi untuk menampilkan confusion matrix:

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(np.argmax(y_true, axis=1), np.argmax(y_pred, axis=1))
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=emotions, yticklabels=emotions)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

# Cross Validation
Implementasi k-fold cross validation untuk evaluasi model:

In [ ]:
num_classes = len(emotions)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

def train_with_cross_validation(X, y):
    fold_accuracies = []
    best_model = None
    best_val_accuracy = 0
    best_history = None
    
    for fold_no, (train_idx, val_idx) in enumerate(tqdm(kf.split(X), total=5, 
                                                      desc="Cross-Validation", unit="fold")):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        model = create_model(X_train.shape[1], num_classes)
        history = model.fit(X_train, y_train, 
                          epochs=100, 
                          batch_size=64, 
                          validation_data=(X_val, y_val), 
                          verbose=0, 
                          callbacks=[tqdm_callback])
        
        score = model.evaluate(X_val, y_val, verbose=0)
        fold_accuracies.append(score[1])
        
        if score[1] > best_val_accuracy:
            best_val_accuracy = score[1]
            best_model = model
            best_history = history
    
    avg_accuracy = np.mean(fold_accuracies) * 100
    print(f"Average accuracy: {avg_accuracy:.2f}%")
    plot_training_history(best_history)
    return best_model

# Callback Progress
Custom callback untuk menampilkan progress bar selama training:

In [ ]:
from tensorflow.keras.callbacks import Callback

class TqdmCallback(Callback):
    def __init__(self, epochs):
        self.epochs = epochs
        self.pbar = None
    
    def on_train_begin(self, logs=None):
        self.pbar = tqdm(total=self.epochs, desc="Training epochs", unit="epoch")
    
    def on_epoch_end(self, epoch, logs=None):
        self.pbar.update(1)
    
    def on_train_end(self, logs=None):
        self.pbar.close()

tqdm_callback = TqdmCallback(epochs=100)

# Pelatihan Model
Proses utama pelatihan model dengan Gabor features:

In [ ]:
with tqdm(total=1, desc="Training model", unit="model") as pbar:
    gabor_model = train_with_cross_validation(X_gabor, y_gabor)
    pbar.update(1)

# Evaluasi Akhir
Testing model pada data test dan visualisasi hasil:

In [ ]:
with tqdm(total=1, desc="Evaluating model", unit="model") as pbar:
    # Split data
    X_gabor_train, X_gabor_test, y_gabor_train, y_gabor_test = train_test_split(
        X_gabor, y_gabor, test_size=0.2, random_state=42)
    
    # Evaluasi
    gabor_score = gabor_model.evaluate(X_gabor_test, y_gabor_test, verbose=0)
    gabor_pred = gabor_model.predict(X_gabor_test, verbose=0)
    
    # Visualisasi
    plot_confusion_matrix(y_gabor_test, gabor_pred)
    print(f"Final Model Accuracy: {gabor_score[1]*100:.2f}%")
    pbar.update(1)

# Penyimpanan Model
Menyimpan model yang telah dilatih untuk penggunaan selanjutnya:

In [ ]:
with tqdm(total=1, desc="Saving model", unit="model") as pbar:
    gabor_model.save('gabor_emotion_model.h5')
    pbar.update(1)